In [1]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS

In [2]:
pip install langchain_core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\SHRUTI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip freeze

absl-py==2.1.0
aiohttp==3.9.5
aiosignal==1.3.1
altair==5.3.0
annotated-types==0.6.0
anyio==4.3.0
asttokens==2.4.1
astunparse==1.6.3
attrs==23.2.0
backoff==2.2.1
beautifulsoup4==4.12.3
blinker==1.8.2
cachetools==5.4.0
certifi==2024.2.2
chardet==5.2.0
charset-normalizer==3.3.2
click==8.1.7
colorama==0.4.6
comm==0.2.1
contourpy==1.2.0
cycler==0.12.1
dataclasses-json==0.6.7
debugpy==1.8.0
decorator==5.1.1
deepdiff==7.0.1
distro==1.9.0
dm-tree==0.1.8
dnspython==2.6.1
email_validator==2.1.1
emoji==2.12.1
et-xmlfile==1.1.0
executing==2.0.1
faiss-cpu==1.8.0.post1
fastapi==0.110.1
filelock==3.15.4
filetype==1.2.0
firecrawl-py==0.0.20
flatbuffers==24.3.25
fonttools==4.49.0
frozenlist==1.4.1
fsspec==2024.6.1
gast==0.5.4
gitdb==4.0.11
GitPython==3.1.43
google-api-core==2.19.1
google-auth==2.33.0
google-cloud-core==2.4.1
google-cloud-speech==2.27.0
google-cloud-storage==2.18.1
google-crc32c==1.5.0
google-pasta==0.2.0
google-resumable-media==2.7.1
googleapis-common-protos==1.63.2
greenlet==3.0.3
gro

In [4]:
pip install langchain_groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\SHRUTI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
pip install groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\SHRUTI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
#load openAI api key
apikey = os.environ['groq_API_KEY'] = "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"

In [7]:
# Initialise LLM with required params
llm = ChatGroq(model= "llama3-70b-8192", temperature=0, api_key=apikey) 

### (1) Load data

In [8]:
loaders = UnstructuredURLLoader(urls=[
    
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2

### (2) Split data to create chunks

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [10]:
len(docs)

37

In [11]:
docs[0]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nTrending Stocks\n\nOla ElectricÂ\xa0INE0LXG01040, OLAELEC, 544225\n\nHUDCOÂ\xa0INE031A01017, HUDCO, 540530\n\nSuzlon EnergyÂ\xa0INE040H01021, SUZLON, 532667\n\nMazagon DockÂ\xa0INE249Z01012, MAZDOCK, 543237\n\nHind ZincÂ\xa0INE267A01025, HINDZINC, 500188\n\n\n\nQuotes\n\nMutual Funds\n\nCommodities\n\nFutures & Options\n\nCurrency\n\nNews\n\nCryptocurrency\n\nForum\n\nNotices\n\nVideos\n\nGlossary\n\nAll\n\nHello, Login Hello, LoginLog-inor Sign-UpMy AccountMy Profile My PortfolioMy WatchlistFREE Credit Score₹100 Cash RewardFixed DepositsMy AlertsMy MessagesPrice AlertsMy Profile My PROMy PortfolioMy WatchlistFREE Credit Score₹100 Cash RewardFixed DepositsMy AlertsMy MessagesPrice AlertsLogoutChat with UsDownload AppFollow us on:\n\nGo Ad-Free\n\nMy Alerts')

### (3) Create embeddings for these chunks and save them to FAISS index

In [12]:
# Create the embeddings of the chunks using OllamaEmbeddings
embeddings = OllamaEmbeddings(model='mxbai-embed-large')

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

ValueError: Error raised by inference API HTTP code: 500, {"error":"failed to generate embedding"}

In [ ]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

In [ ]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

### (4) Retrieve similar embeddings for a given question and call LLM to retrieve final answer

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002256336FC50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022563C1AA90>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say tha

In [ ]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

chain({"question": query}, return_only_outputs=True)

C:\Users\SHRUTI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
C:\Users\SHRUTI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\SHRUTI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not supp

{'answer': 'FINAL ANSWER: The price of Tiago iCNG is between Rs 6.55 lakh and Rs 8.1 lakh.\n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}